In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#Loading Movies dataset
movies_df=pd.read_csv('/kaggle/input/tmdb-movie-metadata/tmdb_5000_movies.csv')
movies_df

In [ ]:
#Loading Movies dataset
credits_df=pd.read_csv('/kaggle/input/tmdb-movie-metadata/tmdb_5000_credits.csv')
credits_df

In [ ]:

credits_df.columns

In [ ]:
movies_df.columns

In [ ]:
# Renaming the movie_id in th credits_df column
movie_id_renamed = credits_df.rename(index = str,columns = {"movie_id":"id"})


In [ ]:
#Merging both columns to movies dataset
movies_merge = movies_df.merge(movie_id_renamed ,on= 'id')
movies_merge.head()

In [ ]:
movies_df.isnull().sum()

In [ ]:
movies_df.info()

In [ ]:
movies_df.describe()

In [ ]:
cleaned_movies_df=movies_merge.drop(columns=['homepage','title_x','title_y','status','production_countries'])
cleaned_movies_df.head()

In [ ]:
cleaned_movies_df.info()

In [ ]:
cleaned_movies_df.head(1)['overview']

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(min_df = 3, max_features = None,
                      strip_accents="unicode",analyzer = "word",token_pattern = r'\w{1,}',
                      ngram_range=(1, 3),
                      stop_words = "english")

cleaned_movies_df['overview'] =cleaned_movies_df['overview'].fillna('')

In [ ]:
#Now fitting it on the 'overview' text
tfv_matrix = tfidf.fit_transform(cleaned_movies_df['overview'])

In [ ]:
tfv_matrix

In [ ]:
tfv_matrix.shape

In [ ]:
from sklearn.metrics.pairwise import linear_kernel
cosine_sim = linear_kernel(tfv_matrix, tfv_matrix)

In [ ]:
indices = pd.Series(cleaned_movies_df.index, index=cleaned_movies_df['original_title']).drop_duplicates()

In [ ]:
# Movie Recommendation Function
def get_recommendations(title, cosine_sim=cosine_sim):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    # Sorting the movies based on the similarity score
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:11]
    movie_indices = [i[0] for i in sim_scores]
    # Return the top 10 most similar movies
    movies=cleaned_movies_df['original_title'].iloc[movie_indices]
    m_id=cleaned_movies_df['id'].iloc[movie_indices]
    dict={"Movies":movies,"id": m_id}
    final_df=pd.DataFrame(dict)
    final_df.reset_index(drop=True,inplace=True)
    return final_df

In [ ]:
get_recommendations('The Dark Knight Rises')